## Crawling and Extracting Data from Websites

This module uses a set of non-standard libraries, which need to be installed on your machine. By default, your instance should have these installed, but if this is not the case, type these in the Unix shell prompt





In [ ]:
!sudo -H apt-get -y install libxml2-dev libxslt-dev python3-dev

and then

In [ ]:
!sudo -H pip3 install -U lxml

In [ ]:
!sudo -H pip3 install -U pandas

### Fetching the headlines from ESPN.com

Let's start by trying to fetch the headlines from the site ESPN.com.



In [ ]:
import requests # This command allows us to fetch URLs
from lxml import html # This module will allow us to parse the returned HTML/XML
import pandas # To create a dataframe

# Let's start by fetching the page, and parsing it
url = "http://www.espn.com/"
response = requests.get(url) # get the html of that url
doc = html.fromstring(response.text) # parse it and create a document

In [ ]:
doc

Let's start by getting the content of the `<title>` node from the site:

In [ ]:
# This will return back a list of nodes that match
# are called <title>......</title>
results = doc.xpath('//title/text()')
results

In [ ]:
results[0]

The `doc` variable is an `HtmlElement` object, and we can now use **XPath** queries to locate the elements that we need. (Depending on time, we may do in class a tutorial on XPath. For now, you can look at the [W3Schools tutorial](http://www.w3schools.com/xpath/xpath_nodes.asp))

For example, to find all the `<a ...> ... </a>` tags in the returned html, which store the links in the page, we issue the command:

In [ ]:
links = doc.xpath("//a")
len(links)

In [ ]:
links = doc.xpath("//a")
# Iterates over all the links (this means all the nodes
# that matched the //a XPath query) and prints the content
# of the attribute href and the text for that node
for link in links:
    print("==================================")
    print(link.get("href"), "==>", link.text)

In [ ]:
lnk = links[70]
type(lnk)

The `lnk` variable is again an HtmlElement. To get parts of the html element that we need, we can use the `get` method (e.g., to get the `href` attribute) and the `text` method (to get the text within the `<a>...</a>` tag.

In [ ]:
lnk.get("href")

In [ ]:
lnk.text_content()

In [ ]:
lnk.text_content().strip()

Now, let's revisit the _list comprehension_ approach that we discussed in the Python Primer session, for quickly constructing lists:

In [ ]:
urls = [lnk.get("href") for lnk in doc.xpath('//a')]
urls

#### Exercise

Use a list compresension approach, to get the text_content of all the URLs in the page.

In [ ]:
# your code here


And now create a list where we put together text content and the URL for each link

In [ ]:
# your code here


#### Let's get the headlines...

Now, let's examine how we can get the data from the website. The key is to understand the structure of the HTML, where the data that we need is stored, and how to fetch the elements. Then, using the appropriate XPath queries, we will get what we want.

In [ ]:
import requests # This command allows us to fetch URLs
from lxml import html # This module will allow us to parse the returned HTML/XML
import pandas # To create a dataframe

Let's start by fetching the page, and parsing it

In [ ]:
url = "http://www.espn.com/"
response = requests.get(url) # get the html of that url
doc = html.fromstring(response.text) # parse it and create a document

By using the `"Right-Click > Inspect"` option of Chrome,
we right click on the headlines and select `"Inspect"`.
This opens the source code.
There we see that all under a `<div class="headlineStack">` tag.

In [ ]:
headlineNode = doc.xpath('//div[@class="headlineStack"]')

The result of that operation is a list with 6 elements.

In [ ]:
type(headlineNode)

In [ ]:
len(headlineNode)

Each headline is under a  `<li><a href="...."></a>` tag.
So, we get all the `<li><a ...>` tags within the `<div class="headlineStack">`
(which is stored in the "`headlineNode`" variable)

In [ ]:
headlines = headlineNode[1].xpath('.//li/a')
len(headlines)

Now, we have the nodes with the conent in the headlines variable.
We extract the text and the URL.

In [ ]:
data = [{"Title": a.text_content(), "URL": a.get("href")} for a in headlines]
data

And let's create our dataframe, so that we can have a better view

In [ ]:
dataframe = pandas.DataFrame(data)
dataframe

#### Of course, there are always more than one way to skin a cat...

Alternatively, if we did not want to restrict ourselves to just the first headline box, we could write an alternative query, to get back all the headlines, that appear in an XPath `//div[@class="headlineStack"]//li/a`:

In [ ]:
headlines = doc.xpath('//div[@class="headlineStack"]//li/a')
data = [{"Title": a.text_content(), "URL": a.get("href")} for a in headlines]
df = pandas.DataFrame(data)
df

In [ ]:
headlines = doc.xpath('//a[@data-mptype="headline"]')
data = [{"Title": a.text_content(), "URL": a.get("href")} for a in headlines]
df = pandas.DataFrame(data)
df



### In Class Example: Crawl BuzzFeed

* We will try to get the top articles that appear on Buzzfeed
* We will grab the link for the article, the text of the title, the description, and the editor.
* The results will be stored in a dataframe (we will see in detail what a dataframe is, in a couple of modules)


In [ ]:
#your code here
import requests
from lxml import html

resp = requests.get("http://www.buzzfeed.com")
doc = html.fromstring(resp.text)

#### Solution for Buzzfeed (as of October 9, 2018)

In [1]:
import requests # This command allows us to fetch URLs
from lxml import html # This module will allow us to parse the returned HTML/XML
import pandas

# Let's start by fetching the page, and parsing it
url = "http://www.buzzfeed.com/"
response = requests.get(url) # get the html of that url
doc = html.fromstring(response.text) # parse it and create a document

articleNodes = doc.xpath("//div[@data-buzzblock='story-card']") 

def parseArticleNode(article):
    headline = article.xpath(".//h2")[0].text_content()
    headline_link = article.xpath(".//a")[0].get("href")
    description = article.xpath(".//p")[0].text_content()
    editor = article.xpath(".//a[contains(@class,'card__meta__link')]/span")[0].text_content().strip()

    result = {
        "headline": headline,
        "URL" : headline_link,
        "description" : description,
        "editor" : editor
    }
    return result

data = [parseArticleNode(article) for article in articleNodes]
df = pandas.DataFrame(data)
df

,URL,description,editor,headline
0,https://www.buzzfeed.com/emmamcanaw/things-you...,These will *literally* take your hand and show...,Emma McAnaw,27 Things You Need If You've Basically Never D...
1,https://www.buzzfeed.com/newsletters?ref=hplego,We have newsletters to help you stay in the lo...,BuzzFeed,WTF Is Happening Today? BuzzFeed Newsletters C...
2,https://www.buzzfeed.com/ryanschocket2/ariana-...,I can't.,Ryan Schocket,This Is Why Ariana Grande Posted An Instagram ...
3,https://www.buzzfeednews.com/article/mattberma...,The UN ambassador and former South Carolina go...,Matt Berman,Nikki Haley Is Leaving Her Job As Ambassador T...
4,https://www.buzzfeed.com/hermionemellark/choos...,You're never fully dressed without perfume!,HermioneMellark,Choose Between These Aestetically Pleasing Per...
5,https://www.buzzfeed.com/chuphanginichandrakan...,"""Stop! I could've dropped my ________!""",Cee Chandra,Can You Successfully Complete These Iconic Vines?
6,https://www.buzzfeed.com/kylesheng/buzzfeeds-v...,#tryceratops #trymas,Kyle Sheng,The Try Guys Have MERCHANDISE And You Can Own ...
7,https://www.buzzfeednews.com/article/claudiaro...,The singer waded into politics on Sunday with ...,Claudia Rosenbaum,Taylor Swift's Instagram Post Has Caused A Mas...
8,https://www.buzzfeed.com/ellaleighton/plan-you...,Are you straight out of Genovia?,EllaLeighton,Plan Your Perfect Royal Life And We'll Tell Yo...
9,https://www.buzzfeed.com/tessaz88/create-your-...,This quiz is out of this world!,tessaz88,Create Your Own Planet In 9 Steps And We'll Gi...
